In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from ConvBN1d import ConvBN
from LinearBN import LinearBN
from source import train, test

In [2]:
def load_ucr(file):
    data = np.loadtxt(file)
    X = data[:, 1:]
    y = data[:, 0]
    y = np.where(y == 1, 1, 0)  # convert labels to 0/1
    return X, y

# Adjust file paths to your local files
X_train, y_train = load_ucr("../FordA_TRAIN.txt")
X_test, y_test = load_ucr("../FordA_TEST.txt")

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (3601, 500), Test shape: (1320, 500)


In [3]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [4]:
mean = X_train_tensor.mean()
std = X_train_tensor.std()
X_train_tensor = (X_train_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
class LReLU(nn.Module):
    def __init__(self):
        super(LReLU, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(5.0)) 
    def forward(self, x):
        return torch.nn.functional.relu(self.alpha*x)

In [6]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1_out = 16
        self.conv1_size = 15
        self.conv1_padding = 7


        self.conv2_out = 16
        self.conv2_size = 15
        self.conv2_padding = 7

        self.conv3_out = 25
        self.conv3_size = 13
        self.conv3_padding = 6

        self.fc1_out = 2

        self.q = 1e-6
        self.bias_trick_par = nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block

        self.block1 = ConvBN(in_channels=1, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .05, bias_par_init=0.0015)
        self.block2 = ConvBN(in_channels=self.conv1_out, out_channels=self.conv2_out, kernel_size=self.conv2_size, padding=self.conv2_padding, std = .15, bias_par_init=0.0015)
        self.block3 = ConvBN(in_channels=self.conv2_out, out_channels=self.conv3_out, kernel_size=self.conv3_size, padding=self.conv3_padding, std = .15, bias_par_init=0.0015)
               
        
        # torch.manual_seed(0)
        self.w2 = nn.Parameter(torch.randn(self.conv3_out * (500//2//2//2), self.fc1_out))
        nn.init.normal_(self.w2, mean=0.0, std=.6)

        self.dropout = nn.Dropout(0.5)

        self.relu = LReLU()




    def forward(self, x):
        x = F.max_pool1d(self.relu(self.block1(x)), 2)
        x = F.max_pool1d(self.relu(self.block2(x)), 2)
        x = F.max_pool1d(self.relu(self.block3(x)), 2)
        
        x = x.view(x.size(0), -1)
        
        # x = self.relu(self.block3(x))
        # x = self.dropout(x)

        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

    def custom_round(self, n):
        remainder = n % 1000
        base = n - remainder
        if remainder >= 101:
            return base + 1000
        elif remainder <= 100:
            return base

    @torch.no_grad()             # no autograd graph
    def flip_sign_(self, tensor: torch.Tensor, percentage: float) -> torch.Tensor:
        """
        Flip the sign of a random subset of elements *in place*.
    
        Args:
            tensor (torch.Tensor): Any shape, modified in place.
            percentage (float): 0‒1 fraction of elements to flip.
    
        Returns:
            torch.Tensor: The same tensor object (for chaining).
        """
        if percentage <= 0.0:
            return tensor
        if percentage >= 1.0:
            tensor.mul_(-1)
            return tensor                    # all elements flipped
    
        numel = tensor.numel()
        num_to_flip = int(numel * percentage)
        if num_to_flip == 0:
            return tensor
    
        flat = tensor.view(-1)               # view ↔ no copy
        idx = torch.randint(0, numel, (num_to_flip,),
                            device=flat.device)
        flat[idx] *= -1                      # in-place sign change
        return tensor

    def init_hdc(self, ratio, seed, flip_perc=None):
        if not isinstance(ratio, (tuple, int)):
            raise TypeError("ratio must be a tuple of size 4 or and integer")

        elif isinstance(ratio, (int)):
            ratio = (ratio, ratio, ratio, ratio)
            
        if not isinstance(seed, (tuple)):
            raise TypeError("seed must be a tuple of size 4")
        
        self.block1.init_hdc(ratio = ratio[0], seed = seed[0], flip_perc=flip_perc)
        self.block2.init_hdc(ratio = ratio[1], seed = seed[1], flip_perc=flip_perc)
        self.block3.init_hdc(ratio = ratio[2], seed = seed[2], flip_perc=flip_perc)
                
        self.n_last = self.w2.size(0)
        self.nHDC_last = int(self.custom_round(ratio[3] * self.n_last)) if ratio[3]<1000 else int(ratio[3])
        torch.manual_seed(seed[3])
        self.g = (torch.randn(self.w2.size(0), self.nHDC_last, device=self.w2.device)).to(torch.half)
        self.wg = torch.sign(torch.matmul(self.g.t(), self.w2.to(torch.half)))

        if flip_perc is not None and flip_perc > 0.0:
            self.flip_sign_(self.wg, flip_perc)


    def hdc(self, x):
        x = F.max_pool1d(self.relu(self.block1.hdc(x)), 2)
        x = F.max_pool1d(self.relu(self.block2.hdc(x)), 2)
        x = F.max_pool1d(self.relu(self.block3.hdc(x)), 2)

        x = x.view(x.size(0), -1)
        
        x = x + self.bias_trick_par
        x = torch.sign(torch.matmul(x.to(torch.half), self.g))

        return x
        
    def classification_layer(self, x):
        x = x @ self.wg
        return x

In [7]:
from tqdm import tqdm
import time
from torch.utils.data import Subset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


torch.cuda.empty_cache()
model = Network().to(device)
model.load_state_dict(torch.load('FordA_GNet_Training.pth', weights_only = True))

<All keys matched successfully>

In [8]:
model.to(torch.half).to(device)
model.eval()

n_splits = 20
subset_size = 500
print(f"Subset size per split: {subset_size}")

times = []
num_workers = 2
pin_memory = True

flip_percs = np.arange(0.0, 0.51, 0.05)
accuracies = np.zeros((len(flip_percs), n_splits))
hyperdim = 15_000
for i, perc in enumerate(flip_percs):
    for split_idx in tqdm(range(n_splits)):
        # Random sampling using split_idx as seed
        np.random.seed(split_idx)
        split_indices = np.random.choice(len(test_dataset), size=subset_size, replace=False)
        split_subset = Subset(test_dataset, split_indices)
        split_loader = torch.utils.data.DataLoader(split_subset, batch_size=15, shuffle=False)
        
        torch.manual_seed(split_idx+4)
        random_seeds = tuple(torch.randint(0, 1000, (1,)).item() for _ in range(4))
        torch.cuda.empty_cache()
        
        model.init_hdc(hyperdim, random_seeds, perc)

        correct = 0
        total = 0

        t0 = time.time()
        with torch.no_grad():
            for images, labels in split_loader:
                images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                output = model.hdc(images.to(torch.half))
                output = model.classification_layer(output.to(torch.half))
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        t1 = time.time()

        acc = 100 * correct / total
        dt = t1 - t0

        accuracies[i, split_idx] = acc
        times.append(dt)

    print(f'Flip Perc: {perc}, Avg Acc: {np.mean(accuracies[i]):.2f}%')



Subset size per split: 500


100%|██████████| 20/20 [00:44<00:00,  2.21s/it]


Flip Perc: 0.0, Avg Acc: 91.32%


100%|██████████| 20/20 [00:44<00:00,  2.21s/it]


Flip Perc: 0.05, Avg Acc: 91.08%


...

Flip Perc: 0.4, Avg Acc: 74.36%


100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Flip Perc: 0.45, Avg Acc: 67.28%


100%|██████████| 20/20 [00:44<00:00,  2.22s/it]

Flip Perc: 0.5, Avg Acc: 60.42%


In [9]:
np.mean(accuracies, axis=1), np.std(accuracies, axis=1)

(array([91.32, 91.08, 91.3 , 90.82, 89.68, 88.27, 85.55, 80.78, 74.36,
        67.28, 60.42]),
 array([1.67140659, 1.67140659, 1.51459566, 1.27420563, 1.39341308,
        2.02215232, 2.83116584, 3.46635255, 3.36428298, 3.34986567,
        3.56308855]))

In [10]:
from scipy.io import savemat
savemat('FordA_HDCGNet.mat',{'FordA_HDCGNet':accuracies})